# AWS Simple Storage Service (S3) 101

In [53]:
import xarray as xr
import numpy as np
import pandas as pd
import time, datetime
import sys, os
import boto3, botocore, dotenv
import s3fs, fsspec
import zarr

## Slice data for 5GB limit free tier

In [2]:
org_ds = xr.open_zarr('shared/data/INDIAN_OCEAN_025GRID_DAILY.zarr/')

In [3]:
ds_2020 = org_ds.sel(time='2020')

In [4]:
ds_2020.sel(lat=20, lon=90, method='nearest')

<xarray.Dataset>
Dimensions:          (time: 366)
Coordinates:
    lat              float32 20.0
    lon              float32 90.0
  * time             (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
Data variables: (12/19)
    CHL              (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    CHL_uncertainty  (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    adt              (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    air_temp         (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    curr_dir         (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    curr_speed       (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    ...               ...
    ug_curr          (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    v_curr           (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    v_wind           (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    vg_curr          (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    wind_dir         (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
    wind_speed       (time) float32 dask.array<chunksize=(25,), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-19
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [8]:
ds_2020

<xarray.Dataset>
Dimensions:          (time: 366, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
Data variables: (12/19)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    curr_dir         (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    curr_speed       (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    ...               ...
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
    wind_speed       (time, lat, lon) float32 dask.array<chunksize=(25, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-19
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [6]:
sys.getsizeof(ds_2020)

112

In [12]:
# export sample file

# netcdf
ds_2020.to_netcdf('shared/data/sample_aws/IO_2020.nc')

In [15]:
ds_2020_zarr = ds_2020.copy(deep=True)
for var in ds_2020_zarr.data_vars:
    if 'chunks' in ds_2020_zarr[var].encoding:
        del ds_2020_zarr[var].encoding['chunks']

In [18]:
ds_2020_zarr.chunk({'time': 100, 'lon': ds_2020.lon.shape[0], 'lat': ds_2020.lat.shape[0]}).to_zarr('shared/data/sample_aws/IO_2020.zarr', consolidated=True)

In [ ]:
# create environmental variables for aws using .env file and load with dotenv

In [17]:
dotenv.load_dotenv() # Returns true if file is loaded

True

## Upload to S3 as IAM user

### For non Xarray-supported file types 

In [26]:
def upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)
        
    s3_client = boto3.client('s3',
                             region_name='us-west-1',
                             aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                             aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY']
                            )
    
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except Exception as e:
        print('failed uploading file for some reasons...')
        print(e)

In [52]:
upload_file(
    file_name='/home/jovyan/shared/data/INDIAN_OCEAN_025GRID_DAILY.zarr',
    bucket='data-products-mp'
)

failed uploading file for some reasons...
[Errno 21] Is a directory: '/home/jovyan/shared/data/INDIAN_OCEAN_025GRID_DAILY.zarr'


### For ZARR data format

Xarray supports direct upload to s3, which is convenient for folder-like objects like ZARR

Due to the free tier of this bucket and terminal limitations, we cannot:
* Use Accelerate Endpoint
* Use AWS CLI (Command Line Interface) to transfer data


In [76]:
def upload_s3_zarr(file_path, bucket, object_name=None):
    if not object_name:
        object_name= os.path.basename(file_path)
    
    ds = xr.open_zarr(file_path)
    
    s3_out = s3fs.S3FileSystem(
                key=os.environ['AWS_ACCESS_KEY_ID'],
                secret=os.environ['AWS_SECRET_ACCESS_KEY']
            )
    s3_url = f's3://data-products-mp/{object_name}'
    s3_store = s3fs.S3Map(root=s3_url, s3=s3, check=False)
    
    try:
        print('Writing to ', s3_url)    
        ds.to_zarr(s3_store, mode='w', consolidated=True)
        print('Finish uploading to s3')
    except Exception as e:
        print(e)

In [77]:
%%time
fp = '/home/jovyan/shared/data/INDIAN_OCEAN_025GRID_DAILY.zarr' # no forward slash '/'
upload_s3_zarr(fp, bucket='data-products-mp')

Writing to  s3://data-products-mp/INDIAN_OCEAN_025GRID_DAILY.zarr
Finish uploading to s3
CPU times: user 3min 42s, sys: 50.6 s, total: 4min 33s
Wall time: 3min 42s


In [75]:
ds = xr.open_zarr('/home/jovyan/shared/data/CMIP6_HIST_MONTHLY.zarr')
s3_out = s3fs.S3FileSystem(
            key=os.environ['AWS_ACCESS_KEY_ID'],
            secret=os.environ['AWS_SECRET_ACCESS_KEY']
        )
s3_url = 's3://data-products-mp/CMIP6_HIST_MONTHLY.zarr'
s3_store = s3fs.S3Map(root=s3_url, s3=s3, check=False)
print('Writing to ', s3_url)    
ds.to_zarr(s3_store, mode='w', consolidated=True)
print('Finish uploading to s3')

Writing to  s3://data-products-mp/CMIP6_HIST_MONTHLY.zarr
Finish uploading to s3


In [ ]:
zarr.copy_all(ds, s3_url, storage_options={'key': })

# Access public data from s3

In [2]:
s3 = s3fs.S3FileSystem(anon=True) # no need to sign in
bucket_name = 'data-products-mp'

In [3]:
list_of_files = s3.glob(f'{bucket_name}/**')

In [4]:
list_of_files

['data-products-mp/California_PAM_data.ipynb', 'data-products-mp/IO_2020.nc']

In [7]:
ds = xr.open_dataset('/home/jovyan/shared/data/CMIP6_HIST_DAILY.zarr', engine='zarr')

In [54]:
zarr.open('/home/jovyan/shared/data/CMIP6_HIST_DAILY.zarr/')

<zarr.hierarchy.Group '/'>

In [36]:
s3 = boto3.client('s3',
                 region_name='us-west-1',
                 aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                 aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY']
)

In [37]:
objects = s3.list_objects_v2(Bucket='data-products-mp')

In [38]:
for obj in objects['Contents']:
    print(obj['Key'])

California_PAM_data.ipynb
IO_2020.nc
mask.csv
